In [20]:
#Import needed libraries
import os 
import matplotlib.pyplot as plt
from pathlib import Path 
from numpy import zeros
from numpy import asarray
import numpy as np
import pandas as pd
import cv2
import pydicom
import xml.etree.ElementTree as ET
from lxml import etree
from sklearn.model_selection import train_test_split
from keras import preprocessing
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [2]:
#Import libraries for the model
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image.dataframe_iterator import DataFrameIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.applications.vgg16 import VGG16
from keras.layers import Input, Flatten, Dense
from keras.engine import  Model

In [3]:
#Get dicom images
def get_dicom_dict(anno_dict, limit=0):
    dicomFiles = os.walk('/Users/samanthasullivan/Desktop/Prac_II_Code/saved_files/manifest-1608669183333/Lung-PET-CT-Dx', topdown=False)
    dcm_dict = {}
    class_freq_map = {}
    for root, dir, files in dicomFiles:
        for file in files:
            if file.endswith(".dcm"):
                fullPath = os.path.join(root, file)
                dicomInfo = pydicom.read_file(fullPath, force=True)
                dicomFileIdentifier = dicomInfo.SOPInstanceUID;
                #Only use Dicom images that have Class ID 1.1.7
                if(dicomFileIdentifier in anno_dict):
                    if(dicomInfo.SOPClassUID != '1.2.840.10008.5.1.4.1.1.7'):
                        continue
                    if(dicomInfo.SOPClassUID in class_freq_map):
                        class_freq_map[dicomInfo.SOPClassUID] = class_freq_map[dicomInfo.SOPClassUID] + 1
                    else:
                        class_freq_map[dicomInfo.SOPClassUID] = 1
                    dcm_dict[dicomFileIdentifier] = fullPath
                    if(limit > 0 and len(dcm_dict) > limit):
                        return dcm_dict
                    if(len(dcm_dict)%1000 == 0):
                        print(len(dcm_dict))
                        print(class_freq_map)
    return dcm_dict
    

In [4]:
#Get annotations
def get_anno_dict():
    annotation = os.walk('/Users/samanthasullivan/Desktop/Prac_II_Code/saved_files/other data/Annotation', topdown=False)
    anno = []
    anno_dict = {}
    for root, dir, files in annotation:
        for file in files:
            if file.endswith(".xml"):
                try:
                    anno_full_path = os.path.join(root, file)
                    tree = etree.parse(anno_full_path)
                    anno_label = tree.getroot()
                    #Get labels of A, B, E or G and remove .xml from filepath
                    label = anno_label.find("object").find("name").text
                    anno_id = file.replace('.xml', '')
                    anno_dict[anno_id] = label
                #show bad file and continue    
                except:
                    print("Unexpected error!!! Bad file: " + anno_full_path)
                    continue
#                if(len(anno_dict)%500 == 0):
 #                   print(len(anno_dict))
    return anno_dict

In [5]:
#Print all the annotations that met the criteria
anno_dict = get_anno_dict()
print(len(anno_dict))

Unexpected error!!! Bad file: /Users/samanthasullivan/Desktop/Prac_II_Code/saved_files/other data/Annotation/A0024/1.3.6.1.4.1.14519.5.2.1.6655.2359.148117780944430339778694852523.xml
30992


In [6]:
#Print all the Dicom images that have annotations
dcm_dict = get_dicom_dict(anno_dict)
print(len(dcm_dict))

1000
{'1.2.840.10008.5.1.4.1.1.7': 1000}
2000
{'1.2.840.10008.5.1.4.1.1.7': 2000}
3000
{'1.2.840.10008.5.1.4.1.1.7': 3000}
4000
{'1.2.840.10008.5.1.4.1.1.7': 4000}
5000
{'1.2.840.10008.5.1.4.1.1.7': 5000}
6000
{'1.2.840.10008.5.1.4.1.1.7': 6000}
7000
{'1.2.840.10008.5.1.4.1.1.7': 7000}
8000
{'1.2.840.10008.5.1.4.1.1.7': 8000}
9000
{'1.2.840.10008.5.1.4.1.1.7': 9000}
10000
{'1.2.840.10008.5.1.4.1.1.7': 10000}
10357


In [7]:
#Create a class for the Data Image Generator
class DCMDataFrameIterator(DataFrameIterator):
    def __init__(self, *arg, **kwargs):
        self.white_list_formats = ('dcm')
        super(DCMDataFrameIterator, self).__init__(*arg, **kwargs)
        self.dataframe = kwargs['dataframe']
        self.x = self.dataframe[kwargs['x_col']]
        self.y = self.dataframe[kwargs['y_col']]
        self.color_mode = kwargs['color_mode']
        self.target_size = kwargs['target_size']

    def _get_batches_of_transformed_samples(self, indices_array):
        # get batch of images
        print(len(self.x))
        print(indices_array)
        batch_x = np.array([self.read_dcm_as_array(dcm_path, self.target_size, color_mode=self.color_mode)
                            for dcm_path in self.x.iloc[indices_array]])

        batch_y = np.array(self.y.iloc[indices_array].astype(np.uint8)) 

        # transform the images
        if self.image_data_generator is not None:
            for i, (x, y) in enumerate(zip(batch_x, batch_y)):
                transform_params = self.image_data_generator.get_random_transform(x.shape)
                batch_x[i] = self.image_data_generator.apply_transform(x, transform_params)
                 

        return batch_x, batch_y

    @staticmethod
    def read_dcm_as_array(dcm_path, target_size=(256, 256), color_mode='rgb'):
        image_array = pydicom.dcmread(dcm_path).pixel_array
        image_array = cv2.resize(image_array, target_size, interpolation=cv2.INTER_NEAREST) 
        image_array = np.expand_dims(image_array, -1)
        if color_mode == 'rgb':
            image_array = cv2.cvtColor(image_array, cv2.COLOR_GRAY2RGB)
        print(image_array.shape)
        return image_array

In [42]:
#Create an array that holds the correct dcm files that have matching annotations.
anno_labels = []
anno_keys = []
for key in anno_dict.keys():
    if(key in dcm_dict):
        anno_labels.append(anno_dict[key])
        anno_keys.append(key)

#Change the labels to int for the model and print
anno_array = np.array(anno_labels)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(anno_array)
anno_cat = to_categorical(encoded_labels)
print(anno_cat)
anno_encoded_dict = {}
for i in range(0, len(anno_keys)):
    anno_encoded_dict[anno_keys[i]] = anno_cat[i]
    
    
    

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]]


In [41]:
#Create a dataframe for the cleaned and usable data
df = pd.DataFrame({'target':pd.Series(anno_encoded_dict),'image_path':pd.Series(dcm_dict)})
df['image_path'] = df['image_path'].astype('str') 
print(df)




                                                                            target  \
1.3.6.1.4.1.14519.5.2.1.6655.2359.1000366410969...  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]   
1.3.6.1.4.1.14519.5.2.1.6655.2359.1000931701564...  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]   
1.3.6.1.4.1.14519.5.2.1.6655.2359.1000951831341...  [0.0, 0.0, 0.0, 1.0, 0.0, 0.0]   
1.3.6.1.4.1.14519.5.2.1.6655.2359.1001825974274...  [0.0, 0.0, 0.0, 1.0, 0.0, 0.0]   
1.3.6.1.4.1.14519.5.2.1.6655.2359.1002527447226...  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]   
...                                                                            ...   
1.3.6.1.4.1.14519.5.2.1.6655.2359.9984373152522...  [0.0, 0.0, 0.0, 1.0, 0.0, 0.0]   
1.3.6.1.4.1.14519.5.2.1.6655.2359.9984434358737...  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
1.3.6.1.4.1.14519.5.2.1.6655.2359.9984573937225...  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
1.3.6.1.4.1.14519.5.2.1.6655.2359.9986379068768...  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]   
1.3.6.1.4.1.14519.5.2.1.6655.2359.9993480878410...  [0

In [34]:
#Split the data into train and test, train %80 test %20.
train_df, test_df = train_test_split(df, test_size=0.2)
print(len(train_df))
print(len(test_df))


8285
2072


In [35]:
#Define the train and test image size peramiters
train_augmentation_parameters = dict(
    rescale=1.0/255.0,
    rotation_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range = [0.8, 1.2],
    validation_split = 0.2
)

valid_augmentation_parameters = dict(
    rescale=1.0/255.0,
    validation_split = 0.2
)

test_augmentation_parameters = dict(
    rescale=1.0/255.0
)

# The training parameters for the model
BATCH_SIZE = 32
CLASS_MODE = 'input'
COLOR_MODE = 'grayscale'
TARGET_SIZE = (300, 300)
EPOCHS = 10
SEED = 1337

train_consts = {
    'seed': SEED,
    'batch_size': BATCH_SIZE,
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE,  
    'subset': 'training'
}

valid_consts = {
    'seed': SEED,
    'batch_size': BATCH_SIZE,
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE, 
    'subset': 'validation'
}

test_consts = {
    'batch_size': 1,  #size 1 for testing
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE,  # resize input images
    'shuffle': False
}


In [36]:
#Declair the training and validation generators
train_augmenter = ImageDataGenerator(**train_augmentation_parameters)
valid_augmenter = ImageDataGenerator(**valid_augmentation_parameters)



In [37]:
#set the paramiters for the train and validate generators
train_generator = DCMDataFrameIterator(dataframe=train_df,
                                    x_col='image_path',
                                    y_col='target',
                                    image_data_generator=train_augmenter,
                                    **train_consts)

valid_generator = DCMDataFrameIterator(dataframe=test_df,
                             x_col='image_path',
                             y_col='target',
                             image_data_generator=valid_augmenter,
                             **valid_consts)



Found 6628 validated image filenames.
Found 414 validated image filenames.


In [38]:
#Create the model
model = Sequential()

model.add(Conv2D(input_shape=(224,224,3),filters=64,
kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,
kernel_size=(3,3),padding="same", activation="relu"))

model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3),
padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3),
padding="same", activation="relu"))

model.add(Flatten())

model.add(Dense(units=10,activation="relu"))

In [39]:
#Print summary with metrics
model.compile(optimizer= keras.optimizers.RMSprop(lr=0.001), loss='binary_crossentropy',
metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 112, 112, 512)     590336    
_________________________________________________________________
flatten_2 (Flatten)          (None, 6422528)          

In [40]:
#Complete the training
earlyStopping = EarlyStopping(monitor='val_loss', verbose=0, mode='min', patience = 4)

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=len(train_generator),
    callbacks=[earlyStopping],
    epochs=EPOCHS,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)


8285
[ 820 5356 6186  543 3982 3351 4736 3143 4045 4415 5284  613 3533 3703
  248  338 1105 4976 6011 6314 3177  197 1628  327 5774 2204 3826 4533
  173 3960 6379 5104]
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)
(300, 300, 3, 1)


ValueError: setting an array element with a sequence.

In [ ]:
#Plot the model, Not used since model not trained
plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()